In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_6bYEch_C6MyWm9ftZgVbruvc9Tb4j8Ssd8SWZLhB5DYBTEG5orqgMPxZkh5SQ71Uew6fNe'

In [3]:
from pinecone import Pinecone
# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

[
    {
        "name": "textbook",
        "metric": "cosine",
        "host": "textbook-87mf77p.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "wiki",
        "metric": "cosine",
        "host": "wiki-87mf77p.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "test-embe

In [5]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "textbook"
# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)
# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

c:\Users\C113\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'textbook-ns1': {'vector_count': 600}},
 'total_vector_count': 600,
 'vector_type': 'dense'}

In [6]:
from langchain_openai import OpenAIEmbeddings
# OpenAIEmbeddings 객체를 초기화합니다.
# 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
# 질문 리스트를 정의합니다.
question = ["미분적분에 대해 알려줘"]
# 질문을 임베딩하여 벡터로 변환합니다.
embedded_question = embedding.embed_documents(question)
# 임베딩된 질문 벡터를 출력합니다.
print(embedded_question)

[[0.018660344183444977, 0.0018460333812981844, -0.04977533221244812, 0.07719285041093826, -0.02113613672554493, 0.01940632425248623, -0.014876380562782288, -0.03161230683326721, -0.020692871883511543, 0.00465427478775382, -0.036217931658029556, 0.012508700601756573, -0.030769025906920433, 0.022357815876603127, 0.029947364702820778, 0.002679856726899743, -0.025298496708273888, -0.005648916121572256, 0.06443548202514648, -0.013470908626914024, 0.0825120210647583, 0.02196860872209072, -0.06331110745668411, 0.010703209787607193, -0.021936174482107162, -0.042574990540742874, 0.013827682472765446, -0.009497747756540775, 0.05055374652147293, -0.0060705579817295074, -0.055743180215358734, -0.05375389754772186, -0.0010149129666388035, -0.016898097470402718, -0.0072381808422505856, 0.05500801280140877, -0.0063462466932833195, -0.02428223192691803, -0.012551945634186268, 0.022703777998685837, 0.032671816647052765, 0.012227606028318405, -0.01557911653071642, 0.028736496344208717, 0.011081606149673

In [7]:
# Pinecone 인덱스에서 쿼리를 실행합니다.
query_result = index.query(
   namespace="textbook-ns1",  # wiki-ns1 네임스페이스를 지정합니다.
   vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
   top_k=5,  # 상위 5개의 결과를 반환합니다.
   include_vector=False,  # 벡터를 포함하지 않습니다.
   include_metadata=True  # 메타데이터를 포함합니다.
)
# 쿼리 결과에서 각 매치의 ID를 추출하여 리스트로 만듭니다.
result_ids = [r.id for r in query_result.matches]
print(result_ids)  # 결과 ID를 출력합니다.

['93e3d330-e231-428a-a4d6-305563d213c4', 'a8ae8ee8-1bed-4b02-bbcd-20054750fd74', '9bda56d2-9abc-4819-b58e-756e3a7d9904', 'd80d492e-ffee-43c3-b967-40c5f233ec3b', '0c9cf5c5-313f-4cad-b8a0-974d998c2bc0']


In [8]:

# 쿼리 결과에서 각 매치의 점수와 메타데이터를 출력합니다.
for r in query_result.matches:
    print(r.score, r.metadata)

0.337514907 {'chunk_id': 6.0, 'head_chunk': '**Epsilon:** 나는 당신이 채소의 중요성에 대해 많은 열정을 가지고 있다는 것을 알지만, 나는 여전히 그것을 이해할 필요는 없다고 생각합니다. 우리는 단순히 채소가 건강하다고 알고, 그것을 먹으면 된다고 생각합니다. 우리는 그것에 대해 더 많은 것을 알 필요는 없습니다.', 'text': '"채소의 두드러짐은 그것을 먹는 이들의 이해에만 의존하는가?"에 대한 토론 내용:\n\n**Phi:** 인간은 채소의 놀라운 이점에 대해 매우 늦게 깨닫고 있습니다. 왜냐하면 채소는 식단의 하류 계층으로 여겨졌기 때문이죠. 이제 우리는 채소가 우리의 전체적인 건강에 얼마나 필수적인지 깨닫고 있습니다. 하지만 우리가 채소를 이해하는 것은 우리가 그것을 먹는 것만으로 충분하지 않습니다. 우리는 채소의 이점을 홍보하기 위해 노력해야 하고, 채소가 얼마나 우리에게 도움이 되는지에 대해 더 많은 사람들에게 교육해야 합니다.\n\n**Epsilon:** 채소가 건강에 좋다는 것은 사실입니다. 그렇지만 채소를 이해하는 것이 더 중요하다고는 생각하지 않습니다. 왜냐하면 영양을 얻기 위해 채소를 먹는 것만으로 충분하기 때문입니다. 다른 사람들이 채소의 이점을 아는지 혹은 채소를 이해하는지는 중요하지 않습니다. 그러나 나는 많은 사람들이 채소를 좋아하지 않는다는 점에 동의합니다. 따라서 채소가 왜 건강에 좋은지, 그리고 채소가 왜 우리의 식단의 중요한 부분인지 교육하는 것이 중요합니다.\n다음에 수퍼마켓에 가서 채소를 고를 때 더 신중하게 고를 수 있는 능력과 지식을 갖출 수 있습니다.\n\n**Phi:** 그것은 옳은 말입니다. 그러나 우리는 또한 채소를 이해하는 것의 중요성을 인지해야 합니다. 왜냐하면 채소를 이해하면 우리는 그것을 더 잘 감사하고, 그것을 더 많이 먹을 가능성이 높아지기 때문이죠. 예를 들어, 어떤 사람들은 왜 채소가 자신의 건강에 중요한지 모르기 때문에 채소를 먹지 않기를 원합니다

In [9]:
from langchain_pinecone import PineconeVectorStore
# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "full_text"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="head_chunk")
# 질문에 대한 유사도 검색을 수행합니다.
# query는 question 리스트의 첫 번째 요소를 사용하고, 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question[0], k=5, namespace="textbook-ns1")
print(docs)
# 검색된 문서의 메타데이터를 출력합니다.
for doc in docs:
   print(doc.metadata)

[Document(id='93e3d330-e231-428a-a4d6-305563d213c4', metadata={'chunk_id': 6.0, 'text': '"채소의 두드러짐은 그것을 먹는 이들의 이해에만 의존하는가?"에 대한 토론 내용:\n\n**Phi:** 인간은 채소의 놀라운 이점에 대해 매우 늦게 깨닫고 있습니다. 왜냐하면 채소는 식단의 하류 계층으로 여겨졌기 때문이죠. 이제 우리는 채소가 우리의 전체적인 건강에 얼마나 필수적인지 깨닫고 있습니다. 하지만 우리가 채소를 이해하는 것은 우리가 그것을 먹는 것만으로 충분하지 않습니다. 우리는 채소의 이점을 홍보하기 위해 노력해야 하고, 채소가 얼마나 우리에게 도움이 되는지에 대해 더 많은 사람들에게 교육해야 합니다.\n\n**Epsilon:** 채소가 건강에 좋다는 것은 사실입니다. 그렇지만 채소를 이해하는 것이 더 중요하다고는 생각하지 않습니다. 왜냐하면 영양을 얻기 위해 채소를 먹는 것만으로 충분하기 때문입니다. 다른 사람들이 채소의 이점을 아는지 혹은 채소를 이해하는지는 중요하지 않습니다. 그러나 나는 많은 사람들이 채소를 좋아하지 않는다는 점에 동의합니다. 따라서 채소가 왜 건강에 좋은지, 그리고 채소가 왜 우리의 식단의 중요한 부분인지 교육하는 것이 중요합니다.\n다음에 수퍼마켓에 가서 채소를 고를 때 더 신중하게 고를 수 있는 능력과 지식을 갖출 수 있습니다.\n\n**Phi:** 그것은 옳은 말입니다. 그러나 우리는 또한 채소를 이해하는 것의 중요성을 인지해야 합니다. 왜냐하면 채소를 이해하면 우리는 그것을 더 잘 감사하고, 그것을 더 많이 먹을 가능성이 높아지기 때문이죠. 예를 들어, 어떤 사람들은 왜 채소가 자신의 건강에 중요한지 모르기 때문에 채소를 먹지 않기를 원합니다. 그러나 그들이 채소가 암을 예방하고, 면역 체계를 강화하고, 심장 건강을 개선할 수 있다는 사실을 알면 채소를 먹을 가능성이 더 높아질 것입니다.\n\n**Epsilon:** 채소를 이해하는 것은 중요할 수도 있지만, 

 - 프롬프트에서 RAG 결과 활용하기

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm=ChatOpenAI(model='gpt-4.1-mini')


In [11]:
chat_template = ChatPromptTemplate.from_messages([
  ('system','''당신은 교과서 처럼 여러가지 지식을 갖고 있습니다.
   질문에 대해 {context} 활용하여 잘 설명해주세요'''),
  ('user','''{question} 에 대하여 궁금한 지식을 알려주세요.''')
])

In [12]:
context = ""
for doc in docs:
  context += doc.page_content
print(context)

**Epsilon:** 나는 당신이 채소의 중요성에 대해 많은 열정을 가지고 있다는 것을 알지만, 나는 여전히 그것을 이해할 필요는 없다고 생각합니다. 우리는 단순히 채소가 건강하다고 알고, 그것을 먹으면 된다고 생각합니다. 우리는 그것에 대해 더 많은 것을 알 필요는 없습니다."물리적 상태로 정의되는 존재의 성격."에 대한 토론 내용:

**Phi:** 먼저, 현실의 본질은 무엇이라고 생각하세요? 그것은 물리적으로 존재하는 것입니까, 아니면 다른 무엇입니까?

**Epsilon:** 제 생각에는 현실은 물리적인 것과 비물리적인 것 모두의 조합입니다. 물리적 세계는 우리가 볼 수 있고 만질 수 있는 세계이며, 비물리적 세계는 우리**Epsilon:** 채소를 이해하는 것은 중요할 수도 있지만, 그것이 필요하지는 않다고 생각합니다. 우리는 단순히 채소가 건강에 좋다는 것을 알고 있고, 그것을 먹어야 한다는 것을 알고 있으면 충분합니다. 우리는 그것에 대해 너무 많이 생각할 필요가 없습니다.

**Phi:** 그러나 나는 채소를 이해하는 것이 매우 중요하다고 생각합니다. 왜냐하면 채소**Epsilon:** 내 생각엔 작고 보잘것 없는 존재들은 우리에게 더 크고 중요하게 보일 것입니다. 우리가 하늘을 자유롭게 날아다니거나 우리의 존재를 어떤 작은 물체로 확대한다면 우리는 세상을 완전히 다른 관점에서 볼 수 있을 것입니다. 우리는 작은 것들에 더 많은 세부 사항을 볼 수 있고, 그것들이 우리 삶에 미치는 영향을 더 잘 이해할 수 있을 것입**Phi:** 그러나 나는 여전히 채소를 이해하는 것이 매우 중요하다고 생각합니다. 왜냐하면 채소를 이해하면 우리는 그것을 더 잘 감사하고, 그것을 더 많이 먹을 가능성이 높아지기 때문이죠. 예를 들어, 어떤 사람들은 왜 채소가 자신의 건강에 중요한지 모르기 때문에 채소를 먹지 않기를 원합니다. 그러나 그들이 채소가 암을 예방하고, 면역 체계를 강화하고,


In [13]:
prompt = chat_template.format_messages(question=question,context=context)
response = llm.invoke(prompt)
print(response.content)

미분적분학(Calculus)은 수학의 한 분야로, 함수의 변화와 누적량을 연구하는 학문입니다. 크게 두 가지 주요 개념, 즉 **미분(Differentiation)**과 **적분(Integration)**으로 나뉩니다. 이 두 개념은 서로 밀접하게 연결되어 있으며, 자연과학, 공학, 경제학 등 다양한 분야에서 널리 활용됩니다.

---

### 1. 미분 (Differentiation)

- **정의:** 미분은 함수의 변화율을 측정하는 과정입니다. 쉽게 말해, 함수가 어떤 값에서 얼마나 빠르게 변하는지를 나타냅니다.
- **기하학적 의미:** 함수 그래프에서 한 점의 접선의 기울기를 나타냅니다.
- **기호:** 함수 \( f(x) \)의 미분을 \( f'(x) \) 또는 \( \frac{df}{dx} \)로 표시합니다.
- **예시:** 위치 시간 함수가 주어졌을 때, 미분을 통해 속도를 구할 수 있습니다.

#### 미분의 기본 공식
- 상수 함수: \( \frac{d}{dx} c = 0 \)
- 멱함수: \( \frac{d}{dx} x^n = n x^{n-1} \) (n은 실수)
- 합과 차: \( \frac{d}{dx} [f(x) \pm g(x)] = f'(x) \pm g'(x) \)
- 곱의 미분법: \( \frac{d}{dx} [f(x) g(x)] = f'(x)g(x) + f(x) g'(x) \)
- 합성함수 미분법(연쇄법칙): \( \frac{d}{dx} f(g(x)) = f'(g(x)) \cdot g'(x) \)

---

### 2. 적분 (Integration)

- **정의:** 적분은 함수의 값을 누적하여 면적이나 총량을 계산하는 과정입니다.
- **기하학적 의미:** 그래프 아래 부분의 면적을 나타냅니다.
- **기호:** 부정적분(원시함수)은 \( \int f(x) dx \), 정적분은 \( \int_a^b f(x) dx \)로 표현합니다.
- **용도:** 거리, 면적, 부피, 누적값 등을 계산할 때 사용됩니다.

#### 적분의 

In [ ]:
# 랭체인 안쓸 때, 처음 했던 방식
from openai import OpenAI

In [ ]:
client = OpenAI()
test = client.chat.completions.create(
  model='gpt-4.1-mini',
  messages=[
    {"role":"system","content":'''당신은 교과서 처럼 여러가지 지식을 갖고 있습니다.
   질문에 대해 활용하여 잘 설명해주세요'''},
    {"role":"user","content":'''{question} 에 대하여 궁금한 지식을 알려주세요.''' }
  ],
  temperature=0.1
)